# Análisis de la distribución de la popularidad (número de reseñas en el primer mes) #

El objetivo de este notebook es analizar la distribución del número de reseñas de los juegos en Steam

In [ ]:
import pandas as pd
import seaborn as sns
import json
import gzip
import matplotlib.pyplot as plt
import plotly.express as px
from pathlib import Path

## Carga de datos a DataFrame:
Es necesario leer el json y crear un dataset a patir de la lista asociada a la clave `data`.

In [ ]:
base_dir = Path().resolve().parents[1]
data_dir = base_dir / "data"

# No usa variable de entorno pues la ruta será el archivo final con todos los juegos,
# no partes individuales de cada identificador
ruta = Path(data_dir / f"info_steam_games_3.json.gz")

if not ruta.exists():
    raise FileNotFoundError(f"No se encuentra la ruta: {ruta}")

with gzip.open(data_dir / "info_steam_games_3.json.gz", "rt", encoding="UTF-8") as f:
    data = json.load(f)
    
df = pd.DataFrame(data['data'])

df

## Procesado del DataFrame

Se crea una columna nueva del dataframe por cada clave dentro de `appdetails`

In [ ]:
df = df.join(df["appdetails"].apply(pd.Series))
df

In [ ]:
def get_name(x):
    if isinstance(x,dict):
        return x.get("name")
    else:
        return None

def get_genres(x):
    if not isinstance(x, dict):
        return []
    genres = x.get('genres', [])
    if not isinstance(genres, list):
        return []
    return [g.get('description') for g in genres if isinstance(g, dict)]

def get_categories(x):
    if not isinstance(x,dict):
        return []
    categories = x.get("categories", [])
    if not isinstance(categories, list):
        return []
    return [c.get("description") for c in categories if isinstance(c, dict)]

def is_free(x):
    if not isinstance(x,dict):
        return []
    price_ov = x.get("price_overview", [])
    if not isinstance(price_ov, dict):
        return []
    return price_ov.get("initial") == 0
        

In [ ]:
df["name"] = df["appdetails"].apply(lambda x : get_name(x))
df["free"] = df["appdetails"].apply(lambda x: is_free(x))
df["categories"] = df["appdetails"].apply(lambda x: get_categories(x))
df["genres"] = df["appdetails"].apply(lambda x: get_genres(x))
df.drop(columns=["appdetails"], inplace=True,errors="ignore")

df


Creación de las columnas `recomendaciones_positivas` y `recomendaciones_negativas`.

In [ ]:
df["recomendaciones_positivas"] = df["appreviewhistogram"].apply(lambda x: x.get("rollups").get("recommendations_up") if isinstance(x, dict) & isinstance(x.get("rollups"), dict) else None)
df["recomendaciones_negativas"] = df["appreviewhistogram"].apply(lambda x: x.get("rollups").get("recommendations_down") if isinstance(x, dict) & isinstance(x.get("rollups"), dict) else None)
df.drop(columns=["appreviewhistogram"], inplace = True, errors="ignore")

df.dropna(subset=["recomendaciones_positivas","recomendaciones_negativas"],inplace = True)

Se comprueba que no hay nulos en ninguna de las 2 columnas creadas

In [ ]:
print(f"Nulos en columna recomendaciones positivas: {df["recomendaciones_negativas"].isna().sum()}, Nulos en columna recomendaciones negativas: {df["recomendaciones_positivas"].isna().sum()}")

Por último se ordena el dataset por el número de reseñas totales

In [ ]:
df["total"] = df["recomendaciones_positivas"] + df["recomendaciones_negativas"]

df.sort_values(by="total",ascending=False, inplace=True)

df.head(n=20)

## Gráficos + Conclusiones

#### Histograma de la distribución de la popularidad

In [ ]:
plt.hist(x=df["total"], bins=100)
plt.yscale("log")
plt.xlabel("Numero de reseñas")
plt.ylabel("Número de juegos (log)")
plt.title("Distribución de popularidad (número de reseñas en el primer mes)")

plt.show()

#### Histogramas divididos entre juegos gratuitos y no gratuitos

In [ ]:
fig = px.histogram(df, x = "total",title="Distribución de popularidad (número de reseñas en el primer mes)",
                   labels={"x":"Número de reseñas", "y":"Número de juegos (log)"},nbins= 100,color = "free", opacity= 0.9,facet_col="free")
fig.update_yaxes(type="log")

fig.show()

In [ ]:
print(f"Número de juegos con menos de 50 reseñas en el primer mes: {df[df["total"] < 50].shape[0]}")

#### Análisis para las distintas categorías

##### Boxplot del número de reseñas para distintos géneros (los más populares)

In [ ]:
df_exp = df.explode("genres")

# Se toman los 10 géneros más populares
top_genres = df_exp["genres"].value_counts().head(10).index
top_genres

plt.Figure(figsize=(12,6))

sns.boxplot(data = df_exp[df_exp["genres"].isin(top_genres)],
            x = "genres",
            y = "total")

plt.title("Distribución de la popularidad por género en Steam")
plt.yscale("log")
plt.xticks(rotation = 45)
plt.ylabel("Número de reseñas")
plt.xlabel("Género")

plt.show()

In [ ]:
sns.violinplot(data = df_exp[df_exp["genres"].isin(top_genres)],
               x = "genres",
               y = "total")

plt.title("Distribución de la popularidad por género en Steam")
plt.yscale("log")
plt.xticks(rotation = 45)
plt.ylabel("Número de reseñas")
plt.xlabel("Género")
plt.show()


En ambos gráficos se obtienen las mismas conclusiones. La media del número de reseñas a lo largo de los distintos géneros no parece variar demasiado, además dentro de cada género la distribución sigue siendo de cola larga.

##### Histograma del número de reseñas para los géneros más populares

In [ ]:
fig, ax = plt.subplots(2,5, figsize = (15,8))

# Para convertir en lista que se pueda recorrer con 1 sólo índice
ax = ax.flatten()

for i,genre in enumerate(top_genres):
    data = df_exp[df_exp["genres"] == genre].get("total")

    ax[i].hist(data, bins=30)
    ax[i].set_title(genre)
    ax[i].set_yscale("log")

plt.tight_layout()
plt.show()

Se puede observar como todos los géneros tienen distribuciones muy similares, con gran concentracion en los números de reseñas bajos y una presencia notable de "outliers". Todo esto es coherente con la lógica del mercado de videojuegos ya que por cada género siempre hay algunos títulos que suelen destacar masivamente respecto a los demás; mientras que el grueso se suele localizar en números de reseñas bajos.

#### Conclusiones:

Se aprecia que la distribución era la que esperábamos, una de cola larga con gran concentración de juegos alrededorde números de reseñas bajos y solo unos pocos con muchas reseñas. Además parece que la distribución se mantiene en las distintas categorías (los juegos gratuitos presentan la misma distribución que los no gratuitos) y géneros.

Se puede comprobar si la distribución se mantiene a lo largo de distintas categorías diferentes. -> COMPLETAR